# Extended Euclidean Algorithm and Solving $ax + by = c$

We want to solve the **linear Diophantine equation**

$$
ax + by = c
$$

for integers $x,y$, given integers $a,b,c$.

In this notebook we will:

1. Compute $\gcd(a,b)$.
2. Use the **Extended Euclidean Algorithm** to find integers $(x_0,y_0)$ such that  
   $$
   ax_0 + by_0 = \gcd(a,b).
   $$
3. Decide when a solution to $ax + by = c$ exists.
4. If it exists, scale $(x_0,y_0)$ to get a solution for $c$.
5. Apply everything to the example  
   $$
   993319x + 999331y = 939193.
   $$


## Step 1: When does a solution exist?

A theorem:

- Let $g = \gcd(a,b)$.
- The equation $ax + by = c$ has an integer solution **iff** $g \mid c$.

So compute $g$ and test divisibility.


In [2]:
import math

a = 993319
b = 999331
c = 939193

g = math.gcd(a, b)
g

1

In [3]:
c % g == 0

True

## Step 2: Extended Euclidean Algorithm (finding coefficients as well as the gcd)

The ordinary Euclidean algorithm finds $\gcd(a,b)$ by repeatedly dividing and taking remainders.

Start with

$$
r_0 = a, \qquad r_1 = b
$$

Then apply the division algorithm:

$$
r_0 = q_1 r_1 + r_2
$$

$$
r_1 = q_2 r_2 + r_3
$$

$$
r_2 = q_3 r_3 + r_4
$$

and continue until a remainder becomes 0.  
The **last non-zero remainder** is $\gcd(a,b)$.

---

### The key extra idea

The extended algorithm keeps track of **how each remainder is built from the original numbers** $a$ and $b$.

We maintain numbers $s_i$ and $t_i$ such that

$$
r_i = s_i a + t_i b
$$

This means every remainder is written as a linear combination of the original inputs.

At the start this is easy:

$$
r_0 = a = 1\cdot a + 0\cdot b
$$

$$
r_1 = b = 0\cdot a + 1\cdot b
$$

So initially

$$
s_0=1,\ t_0=0
\qquad
s_1=0,\ t_1=1
$$

---

### General update formulas for the Extended Euclidean Algorithm

At every stage of the algorithm we have two consecutive remainders

$$
r_{i-1}, \qquad r_i
$$

and each of them is written as a linear combination of the original inputs $a$ and $b$:

$$
r_{i-1} = s_{i-1} a + t_{i-1} b
$$

$$
r_i = s_i a + t_i b
$$

---

### Step 1: Compute the next remainder

By the division algorithm, there exists an integer quotient $q_i$ such that

$$
r_{i+1} = r_{i-1} - q_i r_i
$$

---

### Step 2: Substitute the linear-combination expressions

Substitute the formulas for $r_{i-1}$ and $r_i$:

$$
r_{i+1}
=
(s_{i-1} a + t_{i-1} b)
-
q_i (s_i a + t_i b)
$$

---

### Step 3: Factor out $a$ and $b$

$$
r_{i+1}
=
(s_{i-1} - q_i s_i)a
+
(t_{i-1} - q_i t_i)b
$$

---

### Step 4: Read off the new coefficients

Comparing with

$$
r_{i+1} = s_{i+1} a + t_{i+1} b
$$

we obtain the **general update rules**

$$
\boxed{
s_{i+1} = s_{i-1} - q_i s_i
}
$$

$$
\boxed{
t_{i+1} = t_{i-1} - q_i t_i
}
$$

and of course the remainder update

$$
\boxed{
r_{i+1} = r_{i-1} - q_i r_i
}
$$

---

### How this corresponds directly to the code

In the Python implementation we store:

- `old_r` = previous remainder ($r_{i-1}$)
- `r` = current remainder ($r_i$)
- `old_s`, `s` = coefficients of $a$
- `old_t`, `t` = coefficients of $b$
- `q = old_r // r` = quotient in the division step

The update

```python
old_r, r = r, old_r - q*r
old_s, s = s, old_s - q*s
old_t, t = t, old_t - q*t

In [4]:
def extended_gcd(a: int, b: int):
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1

    while r != 0:
        q = old_r // r

        old_r, r = r, old_r - q*r
        old_s, s = s, old_s - q*s
        old_t, t = t, old_t - q*t

    return old_r, old_s, old_t

g, x0, y0 = extended_gcd(a, b)
g, x0, y0

(1, 370178, -367951)

In [5]:
a*x0 + b*y0

1

## Step 3: Scale to solve $ax+by=c$

If

$$
ax_0 + by_0 = g
$$

and

$$
c = mg
$$

then

$$
x = mx_0, \quad y = my_0
$$

is a solution.


In [6]:
m = c // g
x_particular = x0 * m
y_particular = y0 * m

x_particular, y_particular

(347668586354, -345577003543)

In [7]:
a*x_particular + b*y_particular

939193

# Part (b): RSA private key via Extended Euclidean Algorithm

In RSA we choose primes $p,q$ and set

$$
n = pq.
$$

To compute the **private key** we also need the Carmichael totient value

$$
\lambda(n) = \frac{(p-1)(q-1)}{\gcd(p-1,q-1)}.
$$

The public exponent $e$ must satisfy

- $1 < e < \lambda(n)$
- $\gcd(e,\lambda(n)) = 1$

The private exponent $d$ is defined by the congruence

$$
de \equiv 1 \pmod{\lambda(n)}.
$$

That congruence means: $de-1$ is a multiple of $\lambda(n)$, i.e.

$$
de + \lambda(n)k = 1
$$

for some integer $k$.

This is *exactly* a linear Diophantine equation, so we can use the same `extended_gcd`
code from part (a) to compute $d$ as the modular inverse of $e$ modulo $\lambda(n)$.


In [8]:
# Public key data
p = 137
q = 173
n = p * q
e = 7

# Carmichael totient value lambda(n) = lcm(p-1, q-1)
lam = (p - 1) * (q - 1) // math.gcd(p - 1, q - 1)

n, lam

(23701, 5848)

Check the RSA condition $\gcd(e,\lambda(n))=1$ (this guarantees the inverse exists):

In [9]:
math.gcd(e, lam)

1

Now compute $d$ from the equation

$$
de + \lambda(n)k = 1.
$$

If `extended_gcd(e, lam)` returns `(g, x, y)` with

$$
ex + \lambda(n)y = g,
$$

then when $g=1$ we have

$$
ex + \lambda(n)y = 1
\;\;\Rightarrow\;\;
ex \equiv 1 \pmod{\lambda(n)}.
$$

So $x$ is an inverse of $e$ modulo $\lambda(n)$.
We convert it to the standard representative in $\{0,1,\dots,\lambda(n)-1\}$ using `% lam`.


In [10]:
g, x, y = extended_gcd(e, lam)   
g, x, y

(1, 1671, -2)

In [11]:
d = x % lam
d

1671

Verify the defining property $de \equiv 1 \pmod{\lambda(n)}$ in code:

In [12]:
(d * e) % lam

1

So the **private key** is:

- $n = 23701$
- $d = 1671$

(And the public key is $(n,e)=(23701,7)$.)  
Note: $d$ is unique modulo $\lambda(n)$, meaning any $d + k\lambda(n)$ would also work.


# Part (c) (Optional): Encrypt and decrypt a message

RSA encryption (with public key $(n,e)$):

$$
c \equiv m^e \pmod{n}
$$

RSA decryption (with private key $d$):

$$
m \equiv c^d \pmod{n}.
$$

In Python we should **not** compute $m^e$ directly (it can be enormous).  
Instead we use `pow(base, exp, mod)`, which performs fast modular exponentiation.


In [17]:
m = 21051

# Encrypt
ciphertext = pow(m, e, n)

# Decrypt
m_recovered = pow(ciphertext, d, n)

ciphertext, m_recovered

(8839, 21051)

If `m_recovered` equals the original `m`, then encryption/decryption succeeded.